In [30]:
from jupyter_dash import JupyterDash 
from dash import dcc
from dash import html
from dash.dependencies import Input, Output 
import pandas as pd
import plotly.express as px
import dash_bootstrap_components as dbc
from imp import reload

Data Load

In [31]:
clean_files = ['cleaned_data/all_dataset/all_dataset.parquet']
dfs = []
for file in clean_files:
    dfs.append(pd.read_parquet(file))
df = pd.concat(dfs, axis = 0)
df.index = pd.to_datetime(df.index)
df['state_of_charge_percent'] = df['state_of_charge_percent'].clip(0,120)

In [32]:
df['month_name'] = df.index.month_name()

#import data from Étienne
multi_time_cluster_wk = pd.read_csv('cleaned_data/multivariate_time_series_clustering_weekly.csv')

#list_var_rawbox_check = ['battery_voltage', 'current_out', 'state_of_charge_percent', 'temperature', 'power_out']
option_resampling = [{'label':'month','value':'1MS'},{'label':'day','value':'1D'}]

In [33]:
checklist_values = []
for entry in multi_time_cluster_wk.columns:
    if "mean" in entry:
        checklist_values.append(entry)
        

In [34]:
checklist_values

['battery_voltage_mean',
 'current_mean',
 'state_of_charge_percent_mean',
 'temperature_mean']

In [35]:
#!pip install dash-bootstrap-components
#!pip install jupyter-dash

In [36]:
FONT_AWESOME = (
    "https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css"
)

In [37]:
# from plot_functions import median_profiles, plot_times_series, boxplot, timeseriesclustering
from plot_functions import median_profiles, plot_times_series, boxplot, timeseriesclustering
reload(median_profiles)
reload(plot_times_series)
reload(boxplot)

<module 'plot_functions.boxplot' from 'c:\\Users\\egaliounas\\OneDrive - University College London\\Extracurricular\\BatteryDev2022BBoxx\\submission repo\\notebooks\\plot_functions\\boxplot.py'>

In [38]:
options_n_random_batteries = [{'label':f'{c} random batteries','value': c} for c in range(1, df.battery_id.nunique(), 5)]
#option_resampling = [{'label':'month','value':'1MS'},{'label':'day','value':'1D'}]
option_rolling = [{'label':'month','value':'30D'},{'label':'week','value':'7D'},{'label':'day','value':'1D'}]


In [39]:
#Building the app
app = JupyterDash(__name__, external_stylesheets = [dbc.themes.LUX, FONT_AWESOME],
                            meta_tags=[{'name': 'viewport',
                            'content': 'width=device-width, initial-scale=1.0'}]) #if you want to make it adjust when the dashboard is visualized on mobile phones

In [40]:
#Building the layout
app.layout = dbc.Container([
    #------------------------------------------------------------------------------------------------------------------------
                    #Row #1: title row
                    dbc.Row([
                        html.H1('BBoxx Lithium-Ion Battery Field Data Challenge', className = 'text-center mb-4'),
                        html.P('Team 22: Nicole Schauser, Victor Bossard, Etienne Beauchamp, Elias Galiounas, Maria Varini', className = 'text-center m-4')
                    ]), #close row 1
    #------------------------------------------------------------------------------------------------------------------------
                    #Row 2; sub-section title and text
                    dbc.Row([
                        dbc.Col([
                      html.H2('Time series analysis', className = 'text-center mb-4 bg-primary text-white'),
                      html.P("In this section, the raw data provided can be investigated interactively both in the form of a time series and as a statistic (box plot)", className = 'text-center m-4'),
                      html.Label(['y-axis selection']),
                      dcc.Dropdown(id = 'var-dropdown',
                      value = 'voltage',
                       options = [{'label': a, 'value': a} for a in df.columns], className = 'm-1'),
                       #html.Label(['battery_id selection']),
                       #dcc.Dropdown(id = 'id-dropdown',
                        #            value = '1',
                        #            options = [{'label': b, 'value': b} for b in df['battery_id'].unique()], className = 'm-1'),
                        html.Label(['number batteries to plot']),
                        dcc.Dropdown(id = 'n_batteries-dropdown', multi = False, value = 5, options = options_n_random_batteries),
                        html.Label(['rolling aggregation']),
                        dcc.Dropdown(id = 'rolling-dropdown',
                                    value = '7D',
                                    options = option_rolling)
                    ], width = 12)
                    ]), #close row 2
     #------------------------------------------------------------------------------------------------------------------------
                      #Row 3: time series data plots
                    dbc.Row([
                        #dcc.Dropdown(id = 'field_profile-dropdown', multi = False, value = 'net_power_battery', options = options_fields),
                        #dcc.Dropdown(id = 'n_batteries_profile-dropdown', multi = False, value = 5, options = options_n_random_batteries),
                        dcc.Graph(id = 'ts-graph', figure = {})      
                    ]),
    #-------------------------------------------------------------------------------------------------------------------------
                    html.P("As general statistics of the data, the mean daily average and a monthly box plot of the chosen y-axis selection is displayed below.", className = 'text-center m-4'),
    #-------------------------------------------------------------------------------------------------------------------------
                    dbc.Row([
                        #========================================================================================
                        #Column 1 (left)
                        dbc.Col([
                        dcc.Graph(id= 'profile-graph')],
                        width = {'size':5, 'offset': 1}), #close column 1
                        #===========================================================================================
                        #Column 2 (right)
                        dbc.Col([
                            dcc.Graph(id = 'stat-boxplot')
                        ], 
                        width = {'size':5, 'offset': 1})
                        ]), #close Row 3
    #------------------------------------------------------------------------------------------------------------------------
                    #Row 4: sub section title and text
                    dbc.Row([
                        dbc.Col([
                      html.H2('Clustering analysis results', className = 'text-center mb-4 bg-success text-white'),
                      html.P("In this section, the raw data from the battery cycling are grouped with the chosen (relevant) number of clusters. Specifically, the results from the average weekly profile are presented here below for different variables.", className = 'text-center mb-4'),
                      ], width = 12)
                    ]), #close row 4
    #-------------------------------------------------------------------------------------------------------------------------
                    #Row 5: clustering plots
                    dbc.Row([
                        #=======================================================================================
                        #Column 1
                        dbc.Col([
                      html.Label(['y-axis selection']),
                      dcc.Dropdown(id = 'my-checklist',
                      value = 'battery_voltage_mean',
                       options = [{'label': z, 'value': z} for z in checklist_values], className = 'm-1'),
                            html.Div([
                            dcc.Graph(id='avg-week-clusters')
                            ], style = {'padding-left': '150px'}),
                        ], width = 12), # close column #1
                    ]), #close Row 5
    #------------------------------------------------------------------------------------------------------------------------
                #Row 6: sub title and text
                      dbc.Row([
                        dbc.Col([
                      html.H2('Extracting user profiles', className = 'text-center mb-4 bg-info text-white'),
                      html.P("From each of the clusters obtained, a time series of (average) voltage, current, state-of-charge and temperature is derived. These data can be used as input to a battery tester in the lab, to be able to reproduce actual user profiles from the field. These data are available for downloading:", className = 'text-center mb-4'),
                      ], width = 12)
                    ]), #close row 6
    #------------------------------------------------------------------------------------------------------------------------
                #Row 7: User profile plots
                    dbc.Row([
                        #=======================================================================================
                        #Column 1
                        dbc.Col([
                            html.Div([
                            dbc.Button(id='btn',
                            children=[html.I(className="fa fa-download mr-1"), "Download"],
                            n_clicks = 0,
                            color="info",
                            className="mt-1"
                            )
                            ], style = {'padding-left': '550px'}), 
                        
                        dcc.Download(id="download-component"), #this one is an invisible component, you cannot see it on the page
                        ], width = 12), #close column 1
                    ]), #close Row 7
]) #close the dbc container


## Callbacks

In [41]:
@app.callback(
    Output(component_id = 'profile-graph', component_property = 'figure'),
    [
    Input(component_id = 'var-dropdown', component_property = 'value'),
    Input(component_id = 'n_batteries-dropdown', component_property = 'value'),
    ]
)
def callback_plot_median_profile(field, n_batteries):
    return median_profiles.plot(field, n_batteries, df)

In [42]:
@app.callback(
    Output(component_id = 'ts-graph', component_property = 'figure'),
    [
    Input(component_id = 'n_batteries-dropdown', component_property = 'value'),
    Input(component_id = 'var-dropdown', component_property = 'value'),
    Input(component_id = 'rolling-dropdown', component_property = 'value'),
    ]
)
def callback_plot_time_series(n_batteries, selected_var, rolling):
    return plot_times_series.plot(n_batteries, selected_var, rolling, df)

In [43]:
#callback + function for box plot (plot #2)
@app.callback(
    Output(component_id = 'stat-boxplot', component_property = 'figure'),
    [
    Input(component_id = 'var-dropdown', component_property = 'value'),
    #Input(component_id = 'resampling-dropdown', component_property = 'value'), useless => the daily plot is shitty
    ]
)
def callback_plot_boxplot(selected_var, resampling = "1MS"):
    return boxplot.plot(selected_var, resampling, df)


In [44]:
#callback + function for time series clusters (plot #3)
@app.callback(
    Output(component_id = 'avg-week-clusters', component_property = 'figure'),
    [
    Input(component_id = 'my-checklist', component_property = 'value'),
    ]
)
def callback_time_series_clusters(selected_var):
    return timeseriesclustering.plot(selected_var, multi_time_cluster_wk)

In [45]:
#Callback for the download button
@app.callback(
    Output("download-component", "data"),
    Input("btn", "n_clicks"), #The callback function is here triggered by the number of clicks of the button
    prevent_initial_call=True,
)

def func(n_clicks):
    return dcc.send_data_frame(multi_time_cluster_wk.to_csv, "multivariate_time_series_clustering.csv") #This returns a csv file

In [46]:
if __name__ == '__main__':
    app.run_server(debug = True, port = 2000)

Dash app running on http://127.0.0.1:2000/
